In [1]:
import requests
import re
import time
from datetime import datetime, timedelta

fecha_inicio = datetime.strptime("01/01/2025", "%d/%m/%Y")
fecha_fin = datetime.strptime("31/07/2025", "%d/%m/%Y")
fechas = [(fecha_inicio + timedelta(days=i)).strftime("%d/%m/%Y") for i in range((fecha_fin - fecha_inicio).days + 1)]

for fecha in fechas:
    try:
        print("🔍 Procesando fecha:", fecha)
        headers = {
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8',
            'Accept-Language': 'es-ES,es;q=0.7',
            'Cache-Control': 'no-cache',
            'Connection': 'keep-alive',
            'Content-Type': 'application/x-www-form-urlencoded',
            'Origin': 'https://juris.justucuman.gov.ar',
            'Pragma': 'no-cache',
            'Referer': 'https://juris.justucuman.gov.ar/busca_juris_internet_new.php',
            'Sec-Fetch-Dest': 'document',
            'Sec-Fetch-Mode': 'navigate',
            'Sec-Fetch-Site': 'same-origin',
            'Sec-Fetch-User': '?1',
            'Sec-GPC': '1',
            'Upgrade-Insecure-Requests': '1',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36',
            'sec-ch-ua': '"Not)A;Brand";v="8", "Chromium";v="138", "Brave";v="138"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Windows"',
        }

        # 1. Crear sesión
        session = requests.Session()
        session.get("https://juris.justucuman.gov.ar/busca_juris_internet_new.php", headers=headers)

        # 2. Hacer búsqueda
        params_busqueda = {
            'descriptores': '',
            'flagsubmit': '1',
            'buscopc': 'SCT',
            'tribunalo': '00',
            'sentencia': '',
            'fechad': fecha,
            'fechah': fecha,
            'nexpte': '',
            'causa': '',
            'listareg': '',
            'cantsuma': '50',
            'titysuma': 'on',
            'vienede': '1',
            'vistab': '0',
            'inout': '0',
            'estilovista': 'default',
        }
        res_busqueda = session.get("https://juris.justucuman.gov.ar/busca_juris_resultado_tabs_new.php", params=params_busqueda, headers=headers)

        # 3. Extraer registros
        registros = re.findall(r'<input[^>]+name="MyGroup"[^>]+[Vv]alue="(\d{10})"', res_busqueda.text)
        print(f"📄 {len(registros)} registros encontrados:", registros)

        # 4. Agregar uno por uno (simula checkboxes)
        for reg in registros:
            session.get(
                'https://juris.justucuman.gov.ar/administra_seleccionados.php',
                params={
                    'id': reg,
                    'action': 'agrega',
                    'cache': str(int(time.time() * 1000)),
                },
                headers=headers,
            )
            time.sleep(0.1)

        # 5. Confirmar selección (como si se enviara el formulario)
        data_seleccion = [('MyGroup', r) for r in registros] + [
            ('opc1', 'Resalta'),
            ('opc2', 'Relacionado'),
            ('prtexto', 'SoloSumarios'),
        ]
        session.post(
            'https://juris.justucuman.gov.ar/busca_juris_resultado_tabs_new.php',
            params=params_busqueda,
            data=data_seleccion,
            headers=headers,
        )

        # 6. Descargar Word final
        params_word = {
            'cantreg': str(len(registros)),
            'ordensql': 'DESC',
            'buscopc': 'SCT',
            'vistab': '0',
            'titysuma': 'on',
            'datosbuscados': f'{fecha} {fecha}',
            'opc1': 'Resalta',
            'opc2': 'Relacionado',
            'prtexto': 'SoloSumarios',
        }
        res_word = session.get(
            "https://juris.justucuman.gov.ar/imprimir_seleccion_guardar_hv.php",
            params=params_word,
            headers=headers,
        )

        # 7. Guardar como Word
        filename = f"data/fallos_{fecha.replace('/', '-')}.doc"
        with open(filename, "wb") as f:
            f.write(res_word.content)

        print(f"✅ Archivo Word descargado: {filename}")
        print("--------------------------------------------------")
    except Exception as e:
        print(f"❌ Error procesando la fecha {fecha}: {e}")


🔍 Procesando fecha: 01/01/2025
📄 0 registros encontrados: []
✅ Archivo Word descargado: data/fallos_01-01-2025.doc
--------------------------------------------------
🔍 Procesando fecha: 02/01/2025
📄 0 registros encontrados: []
✅ Archivo Word descargado: data/fallos_02-01-2025.doc
--------------------------------------------------
🔍 Procesando fecha: 03/01/2025
📄 12 registros encontrados: ['0007294201', '0007294201', '0007294202', '0007294202', '0007294301', '0007294301', '0007294302', '0007294302', '0007299401', '0007299401', '0007299402', '0007299402']
✅ Archivo Word descargado: data/fallos_03-01-2025.doc
--------------------------------------------------
🔍 Procesando fecha: 04/01/2025
📄 0 registros encontrados: []
✅ Archivo Word descargado: data/fallos_04-01-2025.doc
--------------------------------------------------
🔍 Procesando fecha: 05/01/2025
📄 0 registros encontrados: []
✅ Archivo Word descargado: data/fallos_05-01-2025.doc
--------------------------------------------------
🔍 P